In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, roc_curve, roc_auc_score

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import GlobalMaxPooling1D

In [3]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils  import to_categorical

In [4]:
df=pd.read_csv("Appliance_0.csv")

In [5]:
df.head()

,Occupancy,Appliance ID,Sin(time),Cos(time),Sin(day),Cos(day),Power consumption,Normalized power,P(t)-P(t-1),P(t)-P(t+1),Class Label
0,1,0,0.00000,1.00000,0.0,1.0,0,-0.22789,0,0,0
1,0,0,0.25882,0.96593,0.0,1.0,0,-0.22789,0,0,0
2,1,0,0.50000,0.86603,0.0,1.0,0,-0.22789,0,0,0
3,0,0,0.70711,0.70711,0.0,1.0,0,-0.22789,0,0,0
4,0,0,0.86603,0.50000,0.0,1.0,0,-0.22789,0,-60,0


In [6]:
df.columns

Index(['Occupancy', 'Appliance ID', 'Sin(time)', 'Cos(time)', 'Sin(day)',
       'Cos(day)', 'Power consumption', 'Normalized power', 'P(t)-P(t-1)',
       'P(t)-P(t+1)', 'Class Label'],
      dtype='object')

In [9]:
shuffle(df)

,Occupancy,Appliance ID,Sin(time),Cos(time),Sin(day),Cos(day),Power consumption,Normalized power,P(t)-P(t-1),P(t)-P(t+1),Class Label
4870,0,0,-0.50000,8.660300e-01,-0.32941,-0.944190,0,-0.22789,0,0,0
15211,0,0,-0.96593,2.588200e-01,-0.99511,-0.098820,60,-0.19456,0,0,4
7645,0,0,-0.25882,-9.659300e-01,-0.72364,0.690170,0,-0.22789,0,-60,0
17266,1,0,0.50000,-8.660300e-01,-0.18823,0.982130,0,-0.22789,0,0,0
11166,1,0,1.00000,6.120000e-17,0.98868,-0.150060,60,-0.19456,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3470,0,0,-0.50000,-8.660300e-01,0.61528,-0.788310,60,-0.19456,60,0,1
15478,0,0,-0.50000,8.660300e-01,-0.99592,0.090252,60,-0.19456,0,60,4
5055,1,0,-0.70711,-7.071100e-01,-0.45591,-0.890030,0,-0.22789,0,-60,0
3097,0,0,0.25882,9.659300e-01,0.79618,-0.605060,60,-0.19456,0,0,4


In [10]:
# Split the data into train and test sets
train_size = int(len(df)*0.8)
train, test = df.iloc[:train_size],df.iloc[train_size:]

In [11]:
# Split train and test sets into predictors (X) and outcome (y) variables
x_train = train.drop('Class Label', axis = 1)
y_train = train.loc[:,['Class Label']]
x_test = test.drop('Class Label', axis = 1)
y_test = test.loc[:,['Class Label']]

In [12]:
len(x_train),len(y_train),len(x_test),len(y_test)

(14016, 14016, 3504, 3504)

In [13]:
# Scale the X data 
scaler_x = MinMaxScaler(feature_range = (0,1))
input_scaler = scaler_x.fit(x_train)
train_x_norm = input_scaler.transform(x_train)
test_x_norm = input_scaler.transform(x_test)

In [14]:
# One-hot encode y data
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [15]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((14016, 10), (3504, 10), (14016, 5), (3504, 5))

In [16]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [17]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [18]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((14016, 10, 1), (3504, 10, 1), (14016, 5), (3504, 5))

In [19]:
verbose, epochs, batch_size = 1, 50, 32
n_timesteps, n_features = x_train.shape[1], x_train.shape[2]

In [41]:
# Define the model architecture
model = Sequential()
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(10, 1)))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(5, activation='softmax'))

In [42]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 8, 16)             64        
                                                                 
 conv1d_7 (Conv1D)           (None, 6, 16)             784       
                                                                 
 conv1d_8 (Conv1D)           (None, 4, 16)             784       
                                                                 
 global_max_pooling1d_2 (Glo  (None, 16)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 64)                1088      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                      

In [43]:
#compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
#fit the model
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/50
438/438 [==============================] - 1s 1ms/step - loss: 0.4359 - accuracy: 0.8340
Epoch 2/50
438/438 [==============================] - 0s 1ms/step - loss: 0.0084 - accuracy: 1.0000
Epoch 3/50
438/438 [==============================] - 0s 1ms/step - loss: 7.7085e-04 - accuracy: 1.0000
Epoch 4/50
438/438 [==============================] - 0s 1ms/step - loss: 2.8876e-04 - accuracy: 1.0000
Epoch 5/50
438/438 [==============================] - 0s 1ms/step - loss: 1.6784e-04 - accuracy: 1.0000
Epoch 6/50
438/438 [==============================] - 0s 1ms/step - loss: 1.1620e-04 - accuracy: 1.0000
Epoch 7/50
438/438 [==============================] - 0s 1ms/step - loss: 1.1961e-04 - accuracy: 1.0000
Epoch 8/50
438/438 [==============================] - 0s 1ms/step - loss: 5.8331e-05 - accuracy: 1.0000
Epoch 9/50
438/438 [==============================] - 0s 1ms/step - loss: 5.3919e-05 - accuracy: 1.0000
Epoch 10/50
438/438 [==============================] - 0s 1ms/step - los

In [45]:
_, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=verbose)
#  return accuracy

110/110 [==============================] - 0s 667us/step - loss: 0.0000e+00 - accuracy: 1.0000


In [46]:
accuracy

1.0

In [47]:
prediction = model.predict(x_test)

110/110 [==============================] - 0s 674us/step


In [ ]:
# prediction[1]

In [48]:
y_test

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [49]:
# prediction_NN = np.argmax(prediction[1], axis=-1)

In [50]:
prediction_NN=np.argmax(prediction, axis=-1)

In [51]:
prediction_NN

array([0, 0, 0, ..., 0, 2, 0], dtype=int64)

In [52]:
y_test1 = test.loc[:,['Class Label']]

In [53]:
y_test1=np.array(y_test1)

In [54]:
y_test1

array([[0],
       [0],
       [0],
       ...,
       [0],
       [2],
       [0]], dtype=int64)

In [55]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test1, prediction_NN))

Accuracy =  1.0


In [56]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test1, prediction_NN)
print(cm)

[[2202    0    0    0]
 [   0  290    0    0]
 [   0    0  290    0]
 [   0    0    0  722]]


In [57]:
from sklearn.metrics import classification_report

# print(class_labels)
target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 4']
print(classification_report(y_test1, prediction_NN, target_names=target_names,digits=3))

              precision    recall  f1-score   support

     Class 0      1.000     1.000     1.000      2202
     Class 1      1.000     1.000     1.000       290
     Class 2      1.000     1.000     1.000       290
     Class 4      1.000     1.000     1.000       722

    accuracy                          1.000      3504
   macro avg      1.000     1.000     1.000      3504
weighted avg      1.000     1.000     1.000      3504



In [ ]:
!pip install mlxtend

In [ ]:
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
# class_dict = {0: 'Class 0',
#               1: 'Class 1',
#               2: 'Class 2',
#               3: 'Class 3',
#               4: 'Class 4'}

# multiclass = np.array([[9926,   67,   19,   56,    0],
#                        [178,  855,    0,   41,   40],
#                        [268,    0,  848,    0,    0],
#                        [27,   16,    0, 2326,   15],
#                        [0,    1,    0,   36, 6305]])

# fig, ax = plot_confusion_matrix(
#     conf_mat=multiclass,
#     class_names=class_dict.values(),show_absolute=False,show_normed=True,colorbar=True)